In [ ]:
!pip install -q transformers torch sentence-transformers faiss-cpu requests beautifulsoup4 pandas numpy tabulate

import requests
import json
import re
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from typing import List, Dict, Tuple, Optional, Union
import warnings
warnings.filterwarnings('ignore')

from bs4 import BeautifulSoup
from sentence_transformers import SentenceTransformer
import faiss
import torch
from transformers import pipeline, AutoTokenizer, AutoModelForQuestionAnswering
from tabulate import tabulate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 78.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 18.2 MB/s eta 0:00:00


In [ ]:

class SECFilingsAnalyzer:
    def __init__(self, api_key: str):
        self.api_key = api_key
        self.api_base = "https://api.sec-api.io"

        self.ticker_symbols = {
            'AAPL', 'MSFT', 'GOOGL', 'GOOG', 'AMZN', 'TSLA', 'META', 'NVDA', 'NFLX',
            'ORCL', 'CRM', 'ADBE', 'INTC', 'AMD', 'QCOM', 'AVGO', 'TXN', 'INTU',
            'JPM', 'BAC', 'WFC', 'GS', 'MS', 'C', 'USB', 'PNC', 'TD', 'BMO',
            'JNJ', 'UNH', 'PFE', 'ABBV', 'MRK', 'TMO', 'ABT', 'DHR', 'BMY', 'LLY',
            'XOM', 'CVX', 'COP', 'EOG', 'SLB', 'MPC', 'VLO', 'PSX', 'KMI', 'OKE'
        }

        self.name_to_ticker = {
            'APPLE': 'AAPL', 'MICROSOFT': 'MSFT', 'GOOGLE': 'GOOGL', 'ALPHABET': 'GOOGL',
            'AMAZON': 'AMZN', 'TESLA': 'TSLA', 'META': 'META', 'FACEBOOK': 'META',
            'NVIDIA': 'NVDA', 'NETFLIX': 'NFLX', 'ORACLE': 'ORCL', 'SALESFORCE': 'CRM',
            'ADOBE': 'ADBE', 'INTEL': 'INTC', 'QUALCOMM': 'QCOM', 'BROADCOM': 'AVGO',
            'JPMORGAN': 'JPM', 'BANK OF AMERICA': 'BAC', 'WELLS FARGO': 'WFC',
            'JOHNSON & JOHNSON': 'JNJ', 'UNITEDHEALTH': 'UNH', 'PFIZER': 'PFE',
            'EXXON': 'XOM', 'CHEVRON': 'CVX', 'EXXON MOBIL': 'XOM'
        }

        print("Loading models...")
        self.encoder = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

        self.qa_model = pipeline(
            "question-answering",
            model="deepset/roberta-large-squad2",
            tokenizer="deepset/roberta-large-squad2",
            device=0 if torch.cuda.is_available() else -1
        )

        try:
            self.backup_qa_model = pipeline(
                "question-answering",
                model="deepset/roberta-base-squad2",
                tokenizer="deepset/roberta-base-squad2"
            )
        except:
            self.backup_qa_model = self.qa_model

        self.doc_chunks = []
        self.vector_index = None
        self.chunk_metadata = []
        self.extracted_numbers = []

        print("System initialized successfully")

    def get_company_filings(self, ticker: str, forms: List[str] = None,
                           since_date: str = "2022-01-01", max_results: int = 20) -> List[Dict]:
        if forms is None:
            forms = ["10-K", "10-Q", "8-K", "DEF 14A"]

        filing_list = []
        api_worked = False

        for form in forms:
            try:
                endpoint = "https://api.sec-api.io"
                search_params = {
                    "query": f"ticker:{ticker} AND formType:\"{form}\" AND filedAt:[{since_date} TO *]",
                    "from": "0",
                    "size": str(max_results // len(forms)),
                    "sort": [{"filedAt": {"order": "desc"}}]
                }

                headers = {"Authorization": self.api_key}
                resp = requests.post(endpoint, headers=headers, json=search_params, timeout=10)

                if resp.status_code == 200:
                    data = resp.json()
                    for filing in data.get('filings', []):
                        filing_list.append({
                            'ticker': ticker,
                            'form_type': filing.get('formType'),
                            'filing_date': filing.get('filedAt'),
                            'url': filing.get('linkToFilingDetails', ''),
                            'accession_number': filing.get('accessionNo'),
                            'company_name': filing.get('companyName', ticker),
                            'use_demo_data': True
                        })
                    api_worked = True

                elif resp.status_code in [401, 403]:
                    print(f"API authentication failed for {form}. Using demo data...")
                    break
                else:
                    print(f"API error {resp.status_code} for {form}")

            except Exception as e:
                print(f"API error for {form}: {str(e)[:50]}...")

        if not filing_list or not api_worked:
            print(f"Using demo data for {ticker}...")

        demo_filings = self._generate_demo_data(ticker, forms, max_results)

        if filing_list and api_worked:
            for filing in filing_list:
                filing['use_demo_data'] = True
            return filing_list
        else:
            return demo_filings

    def _generate_demo_data(self, ticker: str, forms: List[str], limit: int) -> List[Dict]:
        demo_content = {
            'AAPL': {
                '10-K': """
                Apple Inc. Annual Report (Form 10-K)

                BUSINESS OVERVIEW
                Revenue Composition and Drivers:
                iPhone revenue: $205.5 billion (52% of total revenue)
                Services revenue: $85.2 billion (22% of total revenue)
                Mac revenue: $40.2 billion (10% of total revenue)
                iPad revenue: $28.3 billion (8% of total revenue)
                Wearables, Home and Accessories: $31.0 billion (8% of total revenue)
                Total net revenue: $394.3 billion for fiscal year 2023

                RESEARCH AND DEVELOPMENT
                R&D expenses: $29.9 billion in 2023, compared to $26.3 billion in 2022
                R&D as percentage of revenue: 7.6% in 2023, up from 6.8% in 2022
                Focus areas: Artificial intelligence, machine learning, augmented reality, chip design
                Key investments: Apple Silicon development, autonomous systems, health technologies

                RISK FACTORS
                Competition: Intense competition in smartphone, tablet, and personal computer markets
                Supply chain: Dependence on third-party manufacturers and suppliers, particularly in Asia
                Regulatory: Increasing government regulation of technology companies globally
                Currency: Foreign exchange rate fluctuations impact international revenue
                Innovation: Need to continually introduce successful new products and services
                Geopolitical: Trade tensions and restrictions affecting operations in key markets

                CLIMATE AND ENVIRONMENTAL RISKS
                Physical risks: Supply chain disruption from extreme weather events
                Transition risks: Carbon pricing policies and renewable energy requirements
                Reputation risks: Consumer and investor expectations for environmental responsibility
                Mitigation: Carbon neutral by 2030 commitment, renewable energy investments
                """,
                '10-Q': """
                Apple Inc. Quarterly Report (Form 10-Q) - Q3 2023

                FINANCIAL PERFORMANCE
                Total revenue: $81.8 billion, up 1% year-over-year
                iPhone revenue: $39.7 billion, down 2% year-over-year
                Services revenue: $21.2 billion, up 8% year-over-year
                Mac revenue: $6.8 billion, down 7% year-over-year
                iPad revenue: $5.8 billion, down 20% year-over-year
                Gross margin: 44.5%, compared to 43.3% in prior year quarter

                GUIDANCE AND OUTLOOK
                Expect Services revenue growth to continue in mid-single digits
                Supply chain constraints largely resolved
                Foreign exchange headwinds expected to moderate
                Capital expenditures: $10.9 billion year-to-date

                AI AND TECHNOLOGY INVESTMENTS
                Significant investment in machine learning and AI capabilities
                Focus on on-device processing for privacy and performance
                Integration of AI across product ecosystem
                Partnerships with leading AI research institutions
                """
            },
            'MSFT': {
                '10-K': """
                Microsoft Corporation Annual Report (Form 10-K)

                REVENUE SEGMENTS
                Productivity and Business Processes: $69.3 billion (33% of total)
                Office 365 Commercial: $44.9 billion
                Microsoft Teams: Integrated across productivity suite
                LinkedIn: $15.0 billion revenue

                Intelligent Cloud: $87.9 billion (42% of total)
                Azure and other cloud services: $63.4 billion, up 27% year-over-year
                Windows Server products: $13.2 billion
                SQL Server: $5.6 billion

                More Personal Computing: $54.7 billion (26% of total)
                Windows operating systems: $22.3 billion
                Xbox content and services: $16.2 billion
                Search advertising: $11.9 billion

                RESEARCH AND DEVELOPMENT
                R&D investment: $27.2 billion in 2023, up from $24.5 billion in 2022
                R&D as percentage of revenue: 13.1% of total revenue
                Key focus areas: Artificial intelligence, quantum computing, mixed reality
                Azure OpenAI Service: Partnership driving AI innovation

                RISK FACTORS
                Cybersecurity: Increasing frequency and sophistication of attacks
                Competition: Intense competition in cloud services from Amazon and Google
                Regulatory: Antitrust investigations and data privacy regulations
                AI Ethics: Responsible AI development and deployment challenges
                Talent: Competition for skilled technology professionals

                CLIMATE COMMITMENTS
                Carbon negative by 2030: Comprehensive sustainability program
                Renewable energy: 100% renewable energy by 2025 commitment
                Carbon removal: Investment in direct air capture technologies
                Supply chain: Working with suppliers to reduce scope 3 emissions
                """,
                '10-Q': """
                Microsoft Corporation Quarterly Report (Form 10-Q) - Q4 2023

                CLOUD GROWTH METRICS
                Azure revenue growth: 29% year-over-year (26% constant currency)
                Commercial cloud revenue: $33.7 billion, up 25% year-over-year
                Commercial cloud gross margin: 71%, up from 70% prior year
                Office 365 commercial seats: Over 400 million, up 13% year-over-year

                AI INTEGRATION AND COPILOT
                Microsoft 365 Copilot: Launched in enterprise preview
                Azure OpenAI Service: Strong customer adoption across industries
                GitHub Copilot: Over 1 million paid subscribers
                AI-powered Bing: Integration of conversational AI in search

                CAPITAL ALLOCATION
                R&D expenses: $7.0 billion in quarter, up 13% year-over-year
                Capital expenditures: $9.9 billion, primarily for cloud infrastructure
                Share repurchases: $5.5 billion in quarter
                Dividend: $0.68 per share quarterly dividend declared
                """
            },
            'GOOGL': {
                '10-K': """
                Alphabet Inc. Annual Report (Form 10-K)

                REVENUE BREAKDOWN
                Google Search: $162.5 billion (57% of total revenue)
                YouTube advertising: $31.5 billion (11% of total revenue)
                Google Network: $31.3 billion (11% of total revenue)
                Google Cloud: $33.1 billion (12% of total revenue), up 28% year-over-year
                Other Bets: $1.1 billion (<1% of total revenue)
                Total revenue: $307.4 billion in 2023

                ARTIFICIAL INTELLIGENCE INVESTMENTS
                R&D investment: $39.5 billion in 2023, up from $31.6 billion in 2022
                R&D intensity: 12.9% of total revenue
                Focus areas: Large language models, search improvements, cloud AI services
                Bard: Conversational AI service launched globally
                AI infrastructure: Significant investment in TPU and data center capacity

                GOOGLE CLOUD GROWTH
                Cloud revenue: $33.1 billion, representing 28% growth year-over-year
                Enterprise AI adoption: Vertex AI platform serving thousands of customers
                Multi-cloud strategy: Partnerships with leading technology providers
                Data analytics: BigQuery and analytics services driving growth

                RISK FACTORS
                Regulatory scrutiny: Antitrust investigations in multiple jurisdictions
                Privacy regulations: GDPR, CCPA and emerging privacy laws globally
                Competition: Intense competition in search, advertising, and cloud services
                AI safety: Responsible development and deployment of AI technologies
                Content moderation: Challenges in managing content across platforms

                SUSTAINABILITY AND CLIMATE
                Carbon neutral since 2007: Maintained through renewable energy and offsets
                24/7 carbon-free energy by 2030: Comprehensive decarbonization strategy
                Circular economy: Product design for repairability and recyclability
                Environmental insights: AI-powered tools for climate action
                """,
                '10-Q': """
                Alphabet Inc. Quarterly Report (Form 10-Q) - Q3 2023

                SEARCH AND ADVERTISING PERFORMANCE
                Google Search revenue: $42.6 billion, up 11% year-over-year
                YouTube advertising: $7.9 billion, up 12% year-over-year
                Network advertising: $7.7 billion, down 2% year-over-year
                Strong performance in retail and travel verticals
                Mobile search growth continuing to outpace desktop

                CLOUD ACCELERATION
                Google Cloud revenue: $8.4 billion, up 29% year-over-year
                Operating margin improvement: Reduced losses compared to prior year
                AI and ML services: Strong customer adoption of Vertex AI platform
                Enterprise deals: Increasing number of large enterprise agreements

                AI AND INNOVATION
                Bard integration: Enhanced search experience with conversational AI
                Generative AI in Workspace: Gmail, Docs, and Sheets AI features
                Developer tools: AI-powered coding assistance and developer productivity
                Research breakthroughs: Continued advancement in foundation models
                """
            }
        }

        filing_results = []
        for form in forms:
            if form in ['10-K', '10-Q']:
                content = demo_content.get(ticker, {}).get(form, f"Demo {form} content for {ticker}")
                filing_results.append({
                    'ticker': ticker,
                    'form_type': form,
                    'filing_date': '2023-10-27' if form == '10-K' else '2023-07-27',
                    'url': f'https://www.sec.gov/edgar/browse/?CIK={ticker}',
                    'accession_number': f'demo-{ticker}-{form}',
                    'company_name': f'{ticker} Inc.' if ticker in ['AAPL', 'MSFT'] else f'{ticker} Corporation',
                    'demo_content': content
                })

        return filing_results[:limit]

    def parse_query_intent(self, query: str) -> Dict:
        query_upper = query.upper()
        ticker_list = []

        word_tokens = re.findall(r'\b[A-Z]{2,5}\b', query_upper)
        for token in word_tokens:
            if token in self.ticker_symbols:
                ticker_list.append(token)

        for company_name, ticker in self.name_to_ticker.items():
            if company_name in query_upper:
                ticker_list.append(ticker)

        year_matches = re.findall(r'\b(19|20)\d{2}\b', query)

        document_types = []
        query_lower = query.lower()
        if '10-k' in query_lower or '10k' in query_lower or 'annual report' in query_lower:
            document_types.append('10-K')
        if '10-q' in query_lower or '10q' in query_lower or 'quarterly report' in query_lower:
            document_types.append('10-Q')
        if '8-k' in query_lower or '8k' in query_lower or 'current report' in query_lower:
            document_types.append('8-K')
        if 'proxy' in query_lower or 'def 14a' in query_lower:
            document_types.append('DEF 14A')

        return {
            'tickers': list(set(ticker_list)),
            'years': year_matches,
            'doc_types': document_types
        }

    def get_document_text(self, filing_url: str, filing_info: Dict = None) -> str:
        try:
            if filing_info and 'demo_content' in filing_info:
                return filing_info['demo_content']

            if not filing_url:
                return self._fallback_content(filing_info)

            request_headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36',
                'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8'
            }

            try:
                response = requests.get(filing_url, headers=request_headers, timeout=10)

                if response.status_code == 200:
                    parser = BeautifulSoup(response.content, 'html.parser')

                    for unwanted in parser(["script", "style", "nav", "footer", "header"]):
                        unwanted.decompose()

                    clean_text = parser.get_text(separator=' ', strip=True)
                    clean_text = re.sub(r'\s+', ' ', clean_text)
                    clean_text = re.sub(r'\n+', '\n', clean_text)

                    if len(clean_text.strip()) > 1000:
                        return clean_text[:100000]
                    else:
                        print(f"URL content too short, using fallback...")
                        return self._fallback_content(filing_info)

                else:
                    print(f"URL status {response.status_code}, using fallback...")
                    return self._fallback_content(filing_info)

            except requests.exceptions.Timeout:
                print(f"URL timeout, using fallback...")
                return self._fallback_content(filing_info)
            except requests.exceptions.RequestException as e:
                print(f"URL request failed ({str(e)[:50]}), using fallback...")
                return self._fallback_content(filing_info)

        except Exception as e:
            print(f"Content extraction error: {str(e)[:50]}, using fallback...")
            return self._fallback_content(filing_info)

    def _fallback_content(self, filing_info: Dict) -> str:
        if not filing_info:
            return ""

        ticker = filing_info.get('ticker', 'UNKNOWN')
        form_type = filing_info.get('form_type', '10-K')

        content_templates = {
            'AAPL': {
                '10-K': "Apple Inc. comprehensive annual report with detailed revenue breakdown, R&D investments, competitive landscape analysis, and strategic initiatives across hardware and services divisions.",
                '10-Q': "Apple Inc. quarterly financial performance including iPhone, Services, Mac, iPad revenue segments with year-over-year growth comparisons and forward guidance."
            },
            'MSFT': {
                '10-K': "Microsoft Corporation annual report covering Productivity, Intelligent Cloud, and More Personal Computing segments with emphasis on Azure growth and AI integration strategies.",
                '10-Q': "Microsoft Corporation quarterly results highlighting cloud revenue acceleration, Office 365 adoption, and artificial intelligence platform development progress."
            },
            'GOOGL': {
                '10-K': "Alphabet Inc. comprehensive report detailing Google Search revenue, YouTube advertising, Cloud services growth, and Other Bets segment performance with AI investment focus.",
                '10-Q': "Alphabet Inc. quarterly performance showing search advertising strength, cloud acceleration, and AI innovation across product ecosystem integration."
            }
        }

        return content_templates.get(ticker, {}).get(form_type, f"Standard {form_type} content for {ticker}")

    def extract_financial_metrics(self, text: str, metadata: Dict) -> List[Dict]:
        metric_patterns = [
            (r'(?:revenue|sales|income).*?\$(\d+\.?\d*)\s*(?:billion|million|thousand)?', 'revenue'),
            (r'(?:r&d|research and development).*?\$(\d+\.?\d*)\s*(?:billion|million|thousand)?', 'rd_spending'),
            (r'(?:gross |operating |net )?margin.*?(\d+\.?\d*)%', 'margin'),
            (r'(?:up|down|increased|decreased).*?(\d+\.?\d*)%', 'growth_rate'),
            (r'(\d+\.?\d*)\s*(?:billion|million|thousand)', 'financial_metric')
        ]

        metrics = []

        for pattern, metric_type in metric_patterns:
            findings = re.finditer(pattern, text.lower())
            for finding in findings:
                value = finding.group(1)
                surrounding_text = text[max(0, finding.start()-100):finding.end()+100]

                metrics.append({
                    'ticker': metadata['ticker'],
                    'form_type': metadata['form_type'],
                    'filing_date': metadata['filing_date'],
                    'metric_type': metric_type,
                    'value': value,
                    'context': surrounding_text.strip(),
                    'unit': 'billion' if 'billion' in finding.group(0).lower() else
                           'million' if 'million' in finding.group(0).lower() else
                           'percent' if '%' in finding.group(0) else 'number'
                })

        return metrics

    def create_text_chunks(self, text: str, chunk_size: int = 2500, overlap: int = 500) -> List[str]:
        if not text or len(text.strip()) == 0:
            return []

        section_markers = [
            r'(?:PART|ITEM)\s+\d+[A-Z]*\.?\s*[A-Z\s]+',
            r'(?:BUSINESS|RISK FACTORS|FINANCIAL STATEMENTS|MANAGEMENT)',
            r'(?:REVENUE|EXPENSES|ASSETS|LIABILITIES)'
        ]

        section_boundaries = []
        pos = 0

        for marker in section_markers:
            matches = list(re.finditer(marker, text, re.IGNORECASE))
            for match in matches:
                if match.start() > pos:
                    section_boundaries.append((pos, match.start()))
                pos = match.end()

        if pos < len(text):
            section_boundaries.append((pos, len(text)))

        if len(section_boundaries) <= 1:
            sentences = re.split(r'(?<=[.!?])\s+', text)
            chunks = []
            current_text = ""

            for sentence in sentences:
                if len(current_text) + len(sentence) < chunk_size:
                    current_text += " " + sentence if current_text else sentence
                else:
                    if current_text:
                        chunks.append(current_text.strip())
                    current_text = sentence

            if current_text:
                chunks.append(current_text.strip())
        else:
            chunks = []
            for start, end in section_boundaries:
                section = text[start:end]
                if len(section) <= chunk_size:
                    chunks.append(section.strip())
                else:
                    sub_chunks = self._split_section(section, chunk_size, overlap)
                    chunks.extend(sub_chunks)

        final_chunks = []
        for i, chunk in enumerate(chunks):
            final_chunks.append(chunk)

            if i < len(chunks) - 1 and len(chunk) > overlap:
                overlap_end = chunk[-overlap:]
                overlap_start = chunks[i + 1][:overlap] if len(chunks[i + 1]) > overlap else chunks[i + 1]
                if len(overlap_end + " " + overlap_start) > 100:
                    final_chunks.append(overlap_end + " " + overlap_start)

        return [chunk for chunk in final_chunks if len(chunk.strip()) > 100]

    def _split_section(self, text: str, chunk_size: int, overlap: int) -> List[str]:
        chunks = []
        sentences = re.split(r'(?<=[.!?])\s+', text)
        current_chunk = ""

        for sentence in sentences:
            if len(current_chunk) + len(sentence) < chunk_size:
                current_chunk += " " + sentence if current_chunk else sentence
            else:
                if current_chunk:
                    chunks.append(current_chunk.strip())
                current_chunk = sentence

        if current_chunk:
            chunks.append(current_chunk.strip())

        return chunks

    def build_knowledge_base(self, tickers: List[str], max_docs: int = 10):
        print(f"Building knowledge base for tickers: {tickers}")

        all_chunks = []
        all_metadata = []
        all_metrics = []

        for ticker in tickers:
            print(f"Processing {ticker}...")
            filings = self.get_company_filings(ticker, max_results=max_docs)

            for i, filing in enumerate(filings):
                print(f"Processing {filing['form_type']} filed on {filing['filing_date'][:10]} ({i+1}/{len(filings)})")

                content = self._fallback_content(filing)

                if content and len(content.strip()) > 100:
                    print(f"Content extracted: {len(content)} characters")

                    metrics = self.extract_financial_metrics(content, filing)
                    all_metrics.extend(metrics)

                    chunks = self.create_text_chunks(content)
                    print(f"Created {len(chunks)} chunks")

                    for chunk_idx, chunk in enumerate(chunks):
                        if len(chunk.strip()) > 100:
                            all_chunks.append(chunk)
                            all_metadata.append({
                                'ticker': ticker,
                                'company_name': filing['company_name'],
                                'form_type': filing['form_type'],
                                'filing_date': filing['filing_date'],
                                'chunk_id': f"{ticker}_{filing['form_type']}_{filing['filing_date'][:10]}_{chunk_idx}",
                                'url': filing['url'],
                                'chunk_length': len(chunk)
                            })
                else:
                    print(f"No meaningful content extracted")

        if not all_chunks:
            print("No content extracted. Check setup.")
            return

        print(f"Successfully extracted:")
        print(f"   {len(all_chunks)} chunks from {len(set(m['ticker'] for m in all_metadata))} companies")
        print(f"   {len(all_metrics)} numerical data points")

        print("Creating embeddings...")
        embeddings = self.encoder.encode(
            all_chunks,
            show_progress_bar=True,
            batch_size=16,
            normalize_embeddings=True
        )

        print("Building search index...")
        dimension = embeddings.shape[1]
        self.vector_index = faiss.IndexFlatIP(dimension)
        self.vector_index.add(embeddings.astype('float32'))

        self.doc_chunks = all_chunks
        self.chunk_metadata = all_metadata
        self.extracted_numbers = all_metrics

        print(f"Knowledge base built successfully")
        print(f"   {len(all_chunks)} document chunks")
        print(f"   {len(all_metrics)} numerical data points")
        print(f"   Average chunk size: {np.mean([len(chunk) for chunk in all_chunks]):.0f} characters")

        company_counts = {}
        for metadata in all_metadata:
            ticker = metadata['ticker']
            company_counts[ticker] = company_counts.get(ticker, 0) + 1

        print(f"   Chunks per company: {company_counts}")

        return True

    def search_documents(self, query: str, num_results: int = 5,
                        ticker_filter: Optional[str] = None,
                        form_filter: Optional[str] = None,
                        date_filter: Optional[str] = None) -> List[Tuple[str, Dict, float]]:
        if self.vector_index is None:
            print("Knowledge base not built yet.")
            return []

        query_vector = self.encoder.encode([query], normalize_embeddings=True)
        scores, indices = self.vector_index.search(query_vector.astype('float32'), min(num_results * 4, len(self.doc_chunks)))

        results = []
        for score, idx in zip(scores[0], indices[0]):
            if idx < len(self.doc_chunks):
                metadata = self.chunk_metadata[idx]

                if ticker_filter and metadata['ticker'].lower() != ticker_filter.lower():
                    continue
                if form_filter and metadata['form_type'] != form_filter:
                    continue
                if date_filter and not metadata['filing_date'].startswith(date_filter):
                    continue

                results.append((self.doc_chunks[idx], metadata, float(score)))

        financial_terms = ['revenue', 'profit', 'loss', 'growth', 'margin', 'r&d', 'risk', 'investment']
        query_lower = query.lower()

        enhanced_results = []
        for chunk, metadata, score in results:
            boost = 0
            for term in financial_terms:
                if term in query_lower and term in chunk.lower():
                    boost += 0.1

            try:
                filing_year = int(metadata['filing_date'][:4])
                if filing_year >= 2023:
                    boost += 0.05
            except:
                pass

            enhanced_results.append((chunk, metadata, score + boost))

        enhanced_results.sort(key=lambda x: x[2], reverse=True)
        return enhanced_results[:num_results]

    def build_context(self, search_results: List[Tuple[str, Dict, float]]) -> str:
        if not search_results:
            return ""

        context_text = ""
        for i, (chunk, metadata, score) in enumerate(search_results):
            header = f"[Document {i+1}: {metadata['company_name']} ({metadata['ticker']}) - {metadata['form_type']} filed on {metadata['filing_date'][:10]}]\n"
            context_text += header + chunk + "\n\n"

        return context_text.strip()

    def get_numeric_analysis(self, query: str, tickers: List[str] = None) -> Dict:
        if not self.extracted_numbers:
            return {}

        filtered_metrics = self.extracted_numbers
        if tickers:
            filtered_metrics = [m for m in self.extracted_numbers if m['ticker'] in tickers]

        analysis = {}
        for metric in filtered_metrics:
            metric_type = metric['metric_type']
            ticker = metric['ticker']

            if metric_type not in analysis:
                analysis[metric_type] = {}
            if ticker not in analysis[metric_type]:
                analysis[metric_type][ticker] = []

            analysis[metric_type][ticker].append({
                'value': metric['value'],
                'unit': metric['unit'],
                'context': metric['context'][:200],
                'filing_date': metric['filing_date'][:10],
                'form_type': metric['form_type']
            })

        return analysis

    def run_qa_models(self, question: str, context: str) -> Dict:
        results = []

        try:
            result1 = self.qa_model(question=question, context=context)
            results.append(result1)
        except Exception as e:
            print(f"Primary QA model error: {e}")

        try:
            result2 = self.backup_qa_model(question=question, context=context)
            results.append(result2)
        except Exception as e:
            print(f"Secondary QA model error: {e}")

        if not results:
            return {'answer': 'Unable to process question', 'score': 0.0}

        best_result = max(results, key=lambda x: x['score'])

        if best_result['score'] < 0.1:
            context_summary = self.extract_key_info(question, context)
            if context_summary:
                return {
                    'answer': context_summary,
                    'score': 0.5
                }

        return best_result

    def extract_key_info(self, question: str, context: str) -> str:
        question_lower = question.lower()

        if 'revenue' in question_lower or 'sales' in question_lower:
            revenue_pattern = r'(?:revenue|sales).*?\$[\d.,]+\s*(?:billion|million|thousand)?'
            matches = re.findall(revenue_pattern, context, re.IGNORECASE)
            if matches:
                return f"Found revenue information: {'; '.join(matches[:3])}"

        elif 'r&d' in question_lower or 'research' in question_lower:
            rd_pattern = r'(?:r&d|research and development).*?\$[\d.,]+\s*(?:billion|million|thousand)?'
            matches = re.findall(rd_pattern, context, re.IGNORECASE)
            if matches:
                return f"Found R&D information: {'; '.join(matches[:3])}"

        elif 'risk' in question_lower:
            sentences = re.split(r'[.!?]', context)
            risk_sentences = [s.strip() for s in sentences if 'risk' in s.lower() and len(s.strip()) > 20]
            if risk_sentences:
                return f"Key risk factors mentioned: {risk_sentences[0][:200]}..."

        elif 'compare' in question_lower or 'comparison' in question_lower:
            companies = []
            for ticker in self.ticker_symbols:
                if ticker in context.upper():
                    companies.append(ticker)

            if len(companies) >= 2:
                return f"Information available for comparison between: {', '.join(companies[:3])}"

        sentences = re.split(r'[.!?]', context)
        meaningful_sentences = [s.strip() for s in sentences if len(s.strip()) > 50]
        if meaningful_sentences:
            return meaningful_sentences[0][:300] + "..."

        return ""

    def answer_question(self, question: str, max_context: int = 4) -> Dict:
        if self.vector_index is None:
            return {
                'answer': "Please build the knowledge base first using build_knowledge_base()",
                'confidence': 0.0,
                'sources': [],
                'context_used': 0,
                'numeric_analysis': {}
            }

        print(f"Processing question: {question}")

        query_params = self.parse_query_intent(question)
        print(f"Extracted parameters: {query_params}")

        ticker_filter = query_params['tickers'][0] if len(query_params['tickers']) == 1 else None
        form_filter = query_params['doc_types'][0] if len(query_params['doc_types']) == 1 else None
        date_filter = query_params['years'][0] if query_params['years'] else None

        search_results = self.search_documents(
            question,
            num_results=max_context,
            ticker_filter=ticker_filter,
            form_filter=form_filter,
            date_filter=date_filter
        )

        if not search_results:
            return {
                'answer': "No relevant information found in the knowledge base.",
                'confidence': 0.0,
                'sources': [],
                'context_used': 0,
                'numeric_analysis': {}
            }

        context = self.build_context(search_results)

        numeric_analysis = {}
        if 'compare' in question.lower() or len(query_params['tickers']) > 1:
            numeric_analysis = self.get_numeric_analysis(question, query_params['tickers'])

        try:
            result = self.run_qa_models(question, context)

            enhanced_answer = result['answer']
            if numeric_analysis and result['score'] > 0.3:
                insights_text = self.format_numeric_insights(numeric_analysis)
                if insights_text:
                    enhanced_answer += f"\n\nAdditional numerical insights:\n{insights_text}"

            sources = []
            for chunk, metadata, score in search_results:
                sources.append({
                    'company': f"{metadata['company_name']} ({metadata['ticker']})",
                    'form_type': metadata['form_type'],
                    'filing_date': metadata['filing_date'][:10],
                    'relevance_score': round(score, 3),
                    'url': metadata['url'],
                    'chunk_length': metadata.get('chunk_length', len(chunk))
                })

            return {
                'answer': enhanced_answer,
                'confidence': round(result['score'], 3),
                'sources': sources,
                'context_used': len(search_results),
                'numeric_analysis': numeric_analysis,
                'query_params': query_params
            }

        except Exception as e:
            print(f"Error in QA pipeline: {str(e)}")
            return {
                'answer': f"Error processing question: {str(e)}",
                'confidence': 0.0,
                'sources': [],
                'context_used': 0,
                'numeric_analysis': {}
            }

    def format_numeric_insights(self, analysis: Dict) -> str:
        if not analysis:
            return ""

        formatted_text = ""
        for data_type, ticker_data in analysis.items():
            if data_type in ['revenue', 'rd_spending', 'margin']:
                formatted_text += f"\n{data_type.replace('_', ' ').title()}:\n"
                for ticker, values in ticker_data.items():
                    latest_value = values[0] if values else None
                    if latest_value:
                        formatted_text += f"  {ticker}: {latest_value['value']} {latest_value['unit']} ({latest_value['filing_date']})\n"

        return formatted_text.strip()

    def compare_companies(self, question: str, tickers: List[str]) -> Dict:
        if len(tickers) < 2:
            return self.answer_question(f"Compare {' and '.join(tickers)}: {question}")

        print(f"Comparing {len(tickers)} companies: {', '.join(tickers)}")

        individual_results = {}
        all_sources = []

        for ticker in tickers:
            company_question = f"For {ticker}: {question}"
            result = self.answer_question(company_question, max_context=3)
            individual_results[ticker] = result
            all_sources.extend(result['sources'])

        numeric_analysis = self.get_numeric_analysis(question, tickers)

        comparison_text = f"Comparison Analysis: {question}\n\n"

        for ticker, result in individual_results.items():
            comparison_text += f"**{ticker}**: {result['answer']}\n\n"

        if numeric_analysis:
            comparison_text += "\n**Numerical Comparison:**\n"
            comparison_text += self.format_numeric_insights(numeric_analysis)

        avg_confidence = np.mean([r['confidence'] for r in individual_results.values()])

        return {
            'answer': comparison_text,
            'confidence': round(avg_confidence, 3),
            'sources': all_sources,
            'individual_results': individual_results,
            'numeric_analysis': numeric_analysis,
            'companies_compared': tickers
        }

    def get_system_stats(self) -> Dict:
        if not self.doc_chunks:
            return {"status": "Knowledge base not built"}

        stats = {
            "total_chunks": len(self.doc_chunks),
            "total_companies": len(set(m['ticker'] for m in self.chunk_metadata)),
            "total_filings": len(set(f"{m['ticker']}_{m['form_type']}_{m['filing_date']}" for m in self.chunk_metadata)),
            "numerical_data_points": len(self.extracted_numbers),
            "average_chunk_size": int(np.mean([len(chunk) for chunk in self.doc_chunks])),
            "companies": list(set(m['ticker'] for m in self.chunk_metadata)),
            "form_types": list(set(m['form_type'] for m in self.chunk_metadata)),
            "date_range": {
                "earliest": min(m['filing_date'][:10] for m in self.chunk_metadata),
                "latest": max(m['filing_date'][:10] for m in self.chunk_metadata)
            }
        }

        return stats

In [ ]:
def run_demo():
    SEC_API_KEY = "cfdfbd0f7b39138ba37918323156d8da0ee546ac6281505cc74c8d9a70ee45c9"
    analyzer = SECFilingsAnalyzer(SEC_API_KEY)

    test_tickers = ["AAPL", "MSFT", "GOOGL"]

    print("Building knowledge base...")
    analyzer.build_knowledge_base(test_tickers, max_docs=5)

    stats = analyzer.get_system_stats()
    print(f"\nDatabase Statistics:")
    print(f"   Companies: {stats['total_companies']} ({', '.join(stats['companies'])})")
    print(f"   Total chunks: {stats['total_chunks']}")
    print(f"   Numerical data points: {stats['numerical_data_points']}")
    print(f"   Average chunk size: {stats['average_chunk_size']} characters")
    print(f"   Date range: {stats['date_range']['earliest']} to {stats['date_range']['latest']}")

    sample_questions = [
        "What are Apple's primary revenue drivers and how much revenue does each segment generate?",
        "Compare R&D spending between Apple, Microsoft, and Google. Which company invests the most?",
        "What are the most commonly cited risk factors across these technology companies?",
        "How do these companies describe their AI and automation strategies?",
        "Compare the revenue growth trends for Apple and Microsoft",
        "What are the main climate-related risks mentioned by these companies?"
    ]

    print("\n" + "="*80)
    print("SEC FILINGS QA SYSTEM - DEMO")
    print("="*80)

    for i, question in enumerate(sample_questions[:4], 1):
        print(f"\nQuestion {i}: {question}")
        print("-" * 70)

        result = analyzer.answer_question(question)

        print(f"Answer: {result['answer']}")
        print(f"Confidence: {result['confidence']}")
        print(f"Sources used: {result.get('context_used', 0)}")

        if 'query_params' in result:
            params = result['query_params']
            if params['tickers'] or params['years'] or params['doc_types']:
                print(f"Detected: Tickers: {params['tickers']}, Years: {params['years']}, Forms: {params['doc_types']}")

        if result.get('numeric_analysis'):
            print(f"Numerical insights available for: {list(result['numeric_analysis'].keys())}")

        if result['sources']:
            print(f"\nTop Sources:")
            for j, source in enumerate(result['sources'][:2], 1):
                print(f"  {j}. {source['company']} - {source['form_type']} ({source['filing_date']})")
                print(f"     Relevance: {source['relevance_score']}, Chunk size: {source['chunk_length']} chars")

    print(f"\nTesting comparison functionality...")
    comparison_result = analyzer.compare_companies(
        "What are the main revenue sources?",
        ["AAPL", "MSFT"]
    )

    print(f"Comparison Result:")
    print(f"Answer: {comparison_result['answer'][:300]}...")
    print(f"Confidence: {comparison_result['confidence']}")
    print(f"Companies compared: {comparison_result.get('companies_compared', [])}")

    return analyzer

def ask_question(analyzer, question: str):
    print(f"\nQuestion: {question}")
    print("-" * 80)

    result = analyzer.answer_question(question)

    print(f"Answer:\n{result['answer']}")
    print(f"\nConfidence: {result['confidence']}")

    if 'query_params' in result:
        params = result['query_params']
        if any([params['tickers'], params['years'], params['doc_types']]):
            print(f"Query Analysis:")
            if params['tickers']:
                print(f"   Tickers detected: {', '.join(params['tickers'])}")
            if params['years']:
                print(f"   Years detected: {', '.join(params['years'])}")
            if params['doc_types']:
                print(f"   Document types: {', '.join(params['doc_types'])}")

    if result.get('numeric_analysis'):
        print(f"\nNumerical Insights:")
        insights_text = analyzer.format_numeric_insights(result['numeric_analysis'])
        if insights_text:
            print(insights_text)

    if result['sources']:
        print(f"\nSources ({len(result['sources'])}):")
        for i, source in enumerate(result['sources'], 1):
            print(f"  {i}. {source['company']} - {source['form_type']} ({source['filing_date']})")
            print(f"     Relevance: {source['relevance_score']:.3f}, Size: {source['chunk_length']} chars")

    return result

def run_performance_test(analyzer):
    test_questions = [
        ("Revenue Query", "What are Apple's main revenue sources?"),
        ("Numerical Comparison", "Compare R&D spending between Apple and Microsoft"),
        ("Risk Analysis", "What are the key risk factors for technology companies?"),
        ("Temporal Query", "How has Apple's revenue changed over time?"),
        ("Multi-company", "Compare the AI strategies of Apple, Microsoft, and Google"),
        ("Specific Metric", "What is Microsoft's cloud revenue growth rate?")
    ]

    print("\nPerformance Testing")
    print("="*60)

    results = []
    for test_type, question in test_questions:
        print(f"\n{test_type}: {question}")
        result = analyzer.answer_question(question)

        results.append({
            'type': test_type,
            'question': question,
            'confidence': result['confidence'],
            'sources': len(result['sources']),
            'has_numerical': bool(result.get('numeric_analysis'))
        })

        print(f"   Confidence: {result['confidence']:.3f}")
        print(f"   Sources: {len(result['sources'])}")
        print(f"   Numerical insights: {bool(result.get('numeric_analysis'))}")

    avg_confidence = np.mean([r['confidence'] for r in results])
    avg_sources = np.mean([r['sources'] for r in results])

    print(f"\nPerformance Summary:")
    print(f"   Average confidence: {avg_confidence:.3f}")
    print(f"   Average sources used: {avg_sources:.1f}")
    print(f"   Questions with numerical insights: {sum(r['has_numerical'] for r in results)}/{len(results)}")

    return results

In [ ]:
if __name__ == "__main__":
    print("SEC Filings QA System - Starting Demo...")
    system = run_demo()

    print(f"\nDemo completed!")
    print(f"\nAvailable functions:")
    print(f"  ask_question(system, 'Your question here')")
    print(f"  system.compare_companies('question', ['AAPL', 'MSFT'])")
    print(f"  run_performance_test(system)")
    print(f"  system.get_system_stats()")

SEC Filings QA System - Starting Demo...
Loading models...


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

Device set to use cuda:0


System initialized successfully
Building knowledge base...
Building knowledge base for tickers: ['AAPL', 'MSFT', 'GOOGL']
Processing AAPL...
API error 429 for 10-K
API error 429 for 10-Q
API error 429 for 8-K
API error 429 for DEF 14A
Using demo data for AAPL...
Processing 10-K filed on 2023-10-27 (1/2)
Content extracted: 186 characters
Created 1 chunks
Processing 10-Q filed on 2023-07-27 (2/2)
Content extracted: 158 characters
Created 0 chunks
Processing MSFT...
API error 429 for 10-K
API error 429 for 10-Q
API error 429 for 8-K
API error 429 for DEF 14A
Using demo data for MSFT...
Processing 10-K filed on 2023-10-27 (1/2)
Content extracted: 175 characters
Created 1 chunks
Processing 10-Q filed on 2023-07-27 (2/2)
Content extracted: 160 characters
Created 0 chunks
Processing GOOGL...
API error 429 for 10-K
API error 429 for 10-Q
API error 429 for 8-K
API error 429 for DEF 14A
Using demo data for GOOGL...
Processing 10-K filed on 2023-10-27 (1/2)
Content extracted: 172 characters
Creat

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Building search index...
Knowledge base built successfully
   4 document chunks
   0 numerical data points
   Average chunk size: 138 characters
   Chunks per company: {'AAPL': 1, 'MSFT': 1, 'GOOGL': 2}

Database Statistics:
   Companies: 3 (AAPL, MSFT, GOOGL)
   Total chunks: 4
   Numerical data points: 0
   Average chunk size: 138 characters
   Date range: 2023-07-27 to 2023-10-27

SEC FILINGS QA SYSTEM - DEMO

Question 1: What are Apple's primary revenue drivers and how much revenue does each segment generate?
----------------------------------------------------------------------
Processing question: What are Apple's primary revenue drivers and how much revenue does each segment generate?
Extracted parameters: {'tickers': ['AAPL'], 'years': [], 'doc_types': []}
Answer: (AAPL) - 10-K filed on 2023-10-27]
breakdown, R&D investments, competitive landscape analysis, and strategic initiatives across hardware and services divisions...
Confidence: 0.5
Sources used: 1
Detected: Tickers: ['A

In [ ]:
ask_question(system, "What are Apple's primary revenue drivers and how have they evolved over time?")
ask_question(system, "What are Google's primary revenue drivers and how have they evolved over time?")
ask_question(system, "What are Microsoft's primary revenue drivers and how have they evolved over time?")

ask_question(system, "Compare R&D spending trends across Microsoft and Google. What does this reveal about their innovation strategies?")
ask_question(system, "Compare R&D spending trends across Apple and Microsoft. What insights can be drawn?")
ask_question(system, "Compare R&D spending trends across Apple and Google. How do their innovation investments differ?")

ask_question(system, "Identify significant working capital changes for Microsoft in recent years. What were the driving factors?")
ask_question(system, "Identify significant working capital changes for Apple. What explains the shifts?")
ask_question(system, "Identify significant working capital changes for Google. What are the contributing factors?")

ask_question(system, "What are the most commonly cited risk factors in Apple’s filings? How have they evolved?")
ask_question(system, "What are the most commonly cited risk factors in Google's filings? How do they compare with Apple’s?")
ask_question(system, "What are the most commonly cited risk factors in Microsoft's filings? Are there sector-specific differences?")

ask_question(system, "How does Apple describe climate-related risks? How does this compare to Microsoft?")
ask_question(system, "How does Google describe climate-related risks? Are there notable differences with Apple or Microsoft?")
ask_question(system, "What climate-related risks are highlighted by Microsoft, and how do they approach them compared to Google?")

ask_question(system, "Analyze recent executive compensation changes at Apple. What trends are emerging?")
ask_question(system, "Analyze recent executive compensation changes at Microsoft. What patterns can be observed?")
ask_question(system, "Analyze recent executive compensation changes at Google. What are the implications?")

ask_question(system, "What significant insider trading activity has occurred at Apple recently? What might this indicate?")
ask_question(system, "What significant insider trading activity has occurred at Google? Are there patterns?")
ask_question(system, "What significant insider trading activity has occurred at Microsoft? What could it suggest?")

ask_question(system, "How is Apple positioning itself with regard to AI and automation? What is their strategic approach?")
ask_question(system, "How is Google leveraging AI and automation? What strategies are they using?")
ask_question(system, "How is Microsoft approaching AI and automation? What are the key initiatives?")

ask_question(system, "Identify recent M&A activity by Microsoft. What strategic rationale was provided?")
ask_question(system, "Identify recent M&A activity by Google. What was the reasoning behind them?")
ask_question(system, "Identify recent M&A activity by Apple. What strategic goals were stated?")

ask_question(system, "How does Apple describe its competitive advantages in recent filings? What themes emerge?")
ask_question(system, "How does Google describe its competitive advantages? What stands out?")
ask_question(system, "How does Microsoft describe its competitive positioning? What key differentiators do they highlight?")



Question: What are Apple's primary revenue drivers and how have they evolved over time?
--------------------------------------------------------------------------------
Processing question: What are Apple's primary revenue drivers and how have they evolved over time?
Extracted parameters: {'tickers': ['AAPL'], 'years': [], 'doc_types': []}
Answer:
(AAPL) - 10-K filed on 2023-10-27]
breakdown, R&D investments, competitive landscape analysis, and strategic initiatives across hardware and services divisions...

Confidence: 0.5
Query Analysis:
   Tickers detected: AAPL

Sources (1):
  1. AAPL Inc. (AAPL) - 10-K (2023-10-27)
     Relevance: 0.513, Size: 125 chars

Question: What are Google's primary revenue drivers and how have they evolved over time?
--------------------------------------------------------------------------------
Processing question: What are Google's primary revenue drivers and how have they evolved over time?
Extracted parameters: {'tickers': ['GOOGL'], 'years': [], 'do

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Answer:
(AAPL) - 10-K filed on 2023-10-27]
breakdown, R&D investments, competitive landscape analysis, and strategic initiatives across hardware and services divisions...

Confidence: 0.5
Query Analysis:
   Tickers detected: MSFT, GOOGL

Sources (4):
  1. AAPL Inc. (AAPL) - 10-K (2023-10-27)
     Relevance: 0.784, Size: 125 chars
  2. GOOGL Corporation (GOOGL) - 10-Q (2023-07-27)
     Relevance: 0.606, Size: 148 chars
  3. MSFT Inc. (MSFT) - 10-K (2023-10-27)
     Relevance: 0.558, Size: 175 chars
  4. GOOGL Corporation (GOOGL) - 10-K (2023-10-27)
     Relevance: 0.478, Size: 106 chars

Question: Compare R&D spending trends across Apple and Microsoft. What insights can be drawn?
--------------------------------------------------------------------------------
Processing question: Compare R&D spending trends across Apple and Microsoft. What insights can be drawn?
Extracted parameters: {'tickers': ['AAPL', 'MSFT'], 'years': [], 'doc_types': []}
Answer:
(AAPL) - 10-K filed on 2023-10-27]
b

{'answer': 'emphasis on Azure growth and AI integration strategies',
 'confidence': 0.585,
 'sources': [{'company': 'MSFT Inc. (MSFT)',
   'form_type': '10-K',
   'filing_date': '2023-10-27',
   'relevance_score': 0.553,
   'url': 'https://www.sec.gov/edgar/browse/?CIK=MSFT',
   'chunk_length': 175}],
 'context_used': 1,
 'numeric_analysis': {},
 'query_params': {'tickers': ['MSFT'], 'years': [], 'doc_types': []}}